# Import modules

In [ ]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

# Parameters for your Pushshift URL
These are probably the most important parameters to consider when building your Pushshift URL:

* size — increase limit of returned entries to 1000
* after — where to start the search
* before — where to end the search
* title — to search only within the submission’s title
* subreddit — to narrow it down to a particular subreddit

In [ ]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?'+'size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

# Extract key information from Submissions

We want key data for further analysis including: 
* Submission Title
* URL 
* Flair
* Author
* Submission post ID
* Score
* Upload Time
* No. of Comments 
* Permalink.


In [ ]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        body = subm['selftext']
    except KeyError:
        body = "NaN" 
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair,body))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

# Update your Search Settings here

only need to change after and before

In [ ]:
#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps
after = "1609459200" #Submissions after this timestamp (1577836800 = 03/01  20)
before = "1612137600" #Submissions before this timestamp (Sun 04 01 2020 00:00:00 GMT-0800 (Pacific Standard Time))
#1606784400
#query = "Cyberpunk 2077" #Keyword(s) to look for in submissions
sub = "berkeley" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [ ]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?size=1000&after=1609459200&before=1612137600&subreddit=berkeley
100
2021-01-04 00:18:02
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1609719482&before=1612137600&subreddit=berkeley
100
2021-01-05 22:01:28
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1609884088&before=1612137600&subreddit=berkeley
100
2021-01-07 20:12:41
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1610050361&before=1612137600&subreddit=berkeley
100
2021-01-10 01:31:28
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1610242288&before=1612137600&subreddit=berkeley
100
2021-01-11 23:01:38
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1610406098&before=1612137600&subreddit=berkeley
100
2021-01-13 07:25:18
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1610522718&before=1612137600&subreddit=berkeley
100
2021-01-15 08:14:44
https://api.pushshift.io/re

# Check your Submission Extraction was successful

In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

1722 submissions have added to list
1st entry is:
Haas Applicant grade submission created: 2021-01-01 02:17:50
Last entry is:
Discipline for gathering created: 2021-01-31 23:57:33


# Save data to CSV file

In [ ]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair","Body"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

input filename of submission file, please add .csv
Jan2021.csv
1722 submissions have been uploaded
